# Accessing and Reading Database/Data Lakehouse Data

**Snowflake demonstration only.**

Use the alternative notebook (**03_Accessing_and_Reading_Database-Data_Lakehouse_Data.ipynb**) if you want to run the code in your environment.

First, install the required R packages if not done already. See [Installing Required R Packages.](../00_Installing_Required_R_Packages.ipynb)

Begin by loading the **reticulate** package which allows users to run Python code, import Python modules, and pass data between R and Python directly within the same R notebook environment.

In [1]:
library(reticulate)

Before we can connect to Snowflake, we need to install the pandas-compatible version of the Snowflake Connector for Python onto the python version used by the reticulate package. First, copy the python path from the first line of output:

In [2]:
py_config()

python:         /home/sas/.cache/R/reticulate/uv/cache/archive-v0/MpTDAGXHW_pWxja4naILa/bin/python3
libpython:      /home/sas/.cache/R/reticulate/uv/python/cpython-3.11.13-linux-x86_64-gnu/lib/libpython3.11.so
pythonhome:     /home/sas/.cache/R/reticulate/uv/cache/archive-v0/MpTDAGXHW_pWxja4naILa:/home/sas/.cache/R/reticulate/uv/cache/archive-v0/MpTDAGXHW_pWxja4naILa
virtualenv:     /home/sas/.cache/R/reticulate/uv/cache/archive-v0/MpTDAGXHW_pWxja4naILa/bin/activate_this.py
version:        3.11.13 (main, Jul 11 2025, 22:43:55) [Clang 20.1.4 ]
numpy:          /home/sas/.cache/R/reticulate/uv/cache/archive-v0/MpTDAGXHW_pWxja4naILa/lib/python3.11/site-packages/numpy
numpy_version:  2.3.1

NOTE: Python version was forced by py_require()

Open a terminal window and run the following line:

<python path from py_config()> -m pip install "snowflake-connector-python[pandas]"

In [4]:
# Import Python modules
json <- import("json")
sf_connector <- import("snowflake.connector")

## Initialize Database Connection Variables

Credentials are stored in the snowflake_cred.json file and need to be accessed. Hard coding credentials should always be avoided.

A sample Snowflake credentials file might look like this:

```json
{
    "account": "<account>",
    "user": "<user>",
    "password": "<password>",
    "database": "<database>",
    "warehouse": "<warehouse>",
    "schema": "<schema>"
}


In [5]:
# Open and load JSON file directly using Python functions via reticulate
open <- import_builtins()$open
f <- open("../../keys/snowflake_cred.json", "r")
sf_credentials_dict <- json$load(f)
f$close()  # Always good practice to close the file


## Connection to Snowflake Database
We open the connection by calling sf.connector.connect() with the parameters set to our credentials.

In [7]:
# Connect to Snowflake using unpacked credentials
conn <- do.call(sf_connector$connect, sf_credentials_dict)
cursor <- conn$cursor()


In [11]:
# Execute query and fetch results
cursor$execute('select * from "customers"')

In [12]:
#Convert to R dataframe
customers_df <- py_to_r(cursor$fetch_pandas_all())

In [13]:
# Clean up
cursor$close()
conn$close()

[1] TRUE

In [14]:
head(customers_df)

,custId,DemHomeOwnerCode,customerGender,EstimatedIncome,regionMedHomeVal,customerSubscrCode,birthDate
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<list>
1,1,H,F,3000,96980,3,7017
2,1000,H,F,105000,316420,3,9076
3,999,H,F,105000,296920,3,NULL
4,998,H,F,105000,203320,3,9620
5,997,H,F,105000,153400,3,6883
6,996,H,F,105000,137020,3,11617


In [15]:
# Convert birthDate from numeric to Date format
customers_df$birthDate[sapply(customers_df$birthDate, function(x) length(x)==0L)] <- NA
customers_df$birthDate <- unlist(customers_df$birthDate, use.names = FALSE)
customers_df$birthDate <- as.Date(customers_df$birthDate)

# View result
head(customers_df)

,custId,DemHomeOwnerCode,customerGender,EstimatedIncome,regionMedHomeVal,customerSubscrCode,birthDate
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<date>
1,1,H,F,3000,96980,3,1989-03-19
2,1000,H,F,105000,316420,3,1994-11-07
3,999,H,F,105000,296920,3,NA
4,998,H,F,105000,203320,3,1996-05-04
5,997,H,F,105000,153400,3,1988-11-05
6,996,H,F,105000,137020,3,2001-10-22


In [16]:
save(customers_df, file = "03_zDEMO_Accessing_and_Reading_Database-Data_Lakehouse_Data.RData")